# Projeto - PIM

In [1]:
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from sklearn import mixture
import os

os.chdir()

img_name = 'Y3.jpeg'

# Abrir Imagem
im = Image.open(img_name)
data = np.asarray(im)

width, height = im.size

#histograma
num, bins, patches = plt.hist(data.flatten(), 255, density=True, facecolor='g', alpha=0.75)

TypeError: chdir() missing required argument 'path' (pos 1)

In [ ]:
lowest_bic = np.infty
bic = []
bestcompo = []
bestcv = []

n_components_range = range(1, 7)
cv_types = ['spherical', 'tied', 'diag', 'full']

s = num.reshape(-1,1)
for n_component in n_components_range:
    for cv_type in cv_types:
        # Fit a mixture of Gaussians with EM
        gmm = mixture.GaussianMixture(n_components=n_component, covariance_type=cv_type)
        gmm.fit(s)
        bic.append(gmm.bic(s))
        if bic[-1] < lowest_bic:
            lowest_bic = bic[-1]
            best_cv=cv_type
            best_ncompo = n_component
            best_gmm = gmm


In [ ]:
import scipy.stats as stats

weights = best_gmm.weights_
means = best_gmm.means_
covars = best_gmm.covariances_

data_gmm = []

if best_cv == "spherical":
    for i in range(best_ncompo):
        data_gmm.append([num*weights[i]*stats.norm.pdf(num,means[i],np.sqrt(covars[i]))])
else:
    for i in range(best_ncompo):
        data_gmm.append([num*weights[i]*stats.norm.pdf(num,means[i],np.sqrt(covars[i][0]))])

data_gmm = np.array(data_gmm).reshape(best_ncompo,255)

In [ ]:
def create_mask(data_gmm):
    mask = []
    for i in range(data_gmm.shape[1]):
        max_data = 0
        x = []
        for j in range(data_gmm.shape[0]):
            if data_gmm[j][i] > max_data:
                k = j
                max_data = data_gmm[j][i]
        
        mask.append(k)

    return mask

mask = create_mask(data_gmm)

ind, = np.where(np.array(mask) == 0)

In [ ]:
new_image = Image.new(mode = "L", size = (width,height), color = 255)

for i in range(height):
    for j in range(width):
        for k in range(len(ind)):
            if data[i][j][0] == ind[k]:
                new_image.putpixel((j, i),0)
                                   
new_image

new_image.save("im"+str(n)+"_tumor.jpeg", 'jpeg')